In [1]:
VALID_PTH = "zindi_data/ValDataset.csv"
PRDICTION_PTH = 'zindi_data/validation/prediction_cond-detr-50_THR0.000_IOU0.800_ID3016.csv'
THR_INF = 0.01
# from zindi_code import CLS_MAPPER
# CLS_MAPPER["NEG"] = 2
# CLS_MAPPER

In [2]:
import pandas as pd
import json
import os
from tqdm import tqdm

import numpy as np

In [3]:
valid_data = pd.read_csv(VALID_PTH)

predictions = pd.read_csv(PRDICTION_PTH)
predictions.sample(3)

,Image_ID,xmin,ymin,w,h,class,confidence,xmax,ymax
287,id_z0i61ad0tq.jpg,637.41,840.16,62.56,64.80,WBC,0.812,699.97,904.96
12755,id_rvsgqs4tyq.jpg,2032.37,1832.31,125.22,150.95,WBC,0.586,2157.59,1983.26
11837,id_u0xldti3vu.jpg,916.39,1423.16,94.10,112.94,Trophozoite,0.057,1010.49,1536.10


In [4]:
valid_data.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,width,height
1979,id_idjqlz4ppb.jpg,WBC,1.0,2394,3245,2605,3435,4032,3016
1865,id_wtp7vfwf0j.jpg,WBC,1.0,653,437,717,513,1920,1080
1645,id_60wk14naw1.jpg,Trophozoite,1.0,1159,1132,1285,1252,4160,3120


In [5]:
# valid_data = valid_data.rename(columns={
# 	"Image_ID": "image_id", "class": "category_id", "confidence": "score", "xmin": "x", "ymin": "y"
# })

# valid_data["w"] = valid_data["xmax"] - valid_data["x"]
# valid_data["h"] = valid_data["ymax"] - valid_data["y"]


# valid_data.sample(3)

In [6]:
predictions["Image_ID"].nunique(), valid_data["Image_ID"].nunique()

(275, 275)

In [7]:
len(set(predictions["Image_ID"].unique()).intersection(valid_data["Image_ID"].unique()))

275

In [8]:
predictions["Image_ID"].value_counts().head(10)

Image_ID
id_15rb0dgq66.jpg    86
id_wij48tbsif.jpg    85
id_qicgns95fp.jpg    85
id_qa70t0m4yc.jpg    85
id_3xidcn77dw.jpg    84
id_ilmjjeqpcy.jpg    83
id_ndhdv3wuvg.jpg    81
id_bwd2w8i876.jpg    79
id_0vaw9rsiro.jpg    78
id_w8xnbd5rvm.jpg    78
Name: count, dtype: int64

In [9]:
valid_data["Image_ID"].value_counts().head(10)

Image_ID
id_15rb0dgq66.jpg    70
id_zg9jc73sf7.jpg    48
id_zdg96srigj.jpg    42
id_ndhdv3wuvg.jpg    39
id_i7nitsscaw.jpg    39
id_z0i61ad0tq.jpg    38
id_ilmjjeqpcy.jpg    37
id_by6e6shi2z.jpg    36
id_q18tfhfneh.jpg    33
id_087dra2apu.jpg    33
Name: count, dtype: int64

In [10]:
predictions["confidence"].describe()

count    14446.000000
mean         0.181902
std          0.195331
min          0.017000
25%          0.051000
50%          0.089000
75%          0.236000
max          0.840000
Name: confidence, dtype: float64

In [11]:
predictions.isna().sum()

Image_ID      0
xmin          0
ymin          0
w             0
h             0
class         0
confidence    0
xmax          0
ymax          0
dtype: int64

In [12]:
len(valid_data[valid_data["class"] == "NEG"])

70

In [13]:
predictions.columns

Index(['Image_ID', 'xmin', 'ymin', 'w', 'h', 'class', 'confidence', 'xmax',
       'ymax'],
      dtype='object')

In [14]:
valid_data.columns

Index(['Image_ID', 'class', 'confidence', 'ymin', 'xmin', 'ymax', 'xmax',
       'width', 'height'],
      dtype='object')

In [15]:
predictions.sample(5)

,Image_ID,xmin,ymin,w,h,class,confidence,xmax,ymax
2816,id_qa70t0m4yc.jpg,1607.97,2921.10,84.54,84.03,Trophozoite,0.320,1692.51,3005.13
6434,id_56cm3hgim3.jpg,879.75,2789.82,101.88,122.49,Trophozoite,0.547,981.63,2912.31
9036,id_igebahf2c3.jpg,712.04,2540.21,103.39,131.09,Trophozoite,0.029,815.43,2671.30
8758,id_u1t3vfc35i.jpg,2269.37,1571.29,142.44,198.28,WBC,0.104,2411.81,1769.57
10122,id_6xbobn7zax.jpg,540.75,166.54,35.12,33.03,Trophozoite,0.046,575.87,199.57


In [ ]:
import torch

image_files: list[str] = valid_data["Image_ID"].unique().tolist()


CLS_MAPPER = {"Trophozoite": 0, "WBC": 0}
keys = ["Trophozoite", "WBC"]


def get_format_pred(file_id: str, data: pd.DataFrame, thrs=THR_INF):
    if isinstance(thrs, float):
        thrs = [thrs] * len(CLS_MAPPER)
    for k, thr in zip(keys, thrs):
        CLS_MAPPER[k] = thr

    pred = data[data["Image_ID"] == file_id].copy()
    pred = pred[pred["class"] != "NEG"]
    pred = pred[
        pred.apply(lambda x: x["confidence"] >= CLS_MAPPER[x["class"]], axis=1)
    ]
    if not len(pred):
        return dict(
            boxes=torch.empty(size=[0, 4], dtype=torch.float).cuda(),
            scores=torch.empty(size=[0], dtype=torch.float).cuda(),
            labels=torch.empty(
                size=[
                    0,
                ],
                dtype=torch.int,
            ).cuda(),
        )
        return dict(
            boxes=torch.tensor([[0.0, 0.0, 0.0, 0.0]], dtype=torch.float).cuda(),
            scores=torch.tensor([1.0], dtype=torch.float).cuda(),
            labels=torch.tensor([CLS_MAPPER["NEG"]], dtype=torch.int).cuda(),
        )
    pred["w"] = pred["xmax"] - pred["xmin"]
    pred["h"] = pred["ymax"] - pred["ymin"]
    boxes = torch.tensor(
        pred[["xmin", "ymin", "w", "h"]].values, dtype=torch.float
    ).cuda()

    scores = torch.tensor(pred["confidence"].values, dtype=torch.float).cuda()
    labels = torch.tensor(
        [CLS_MAPPER[i] for i in pred["class"].values], dtype=torch.int
    ).cuda()
    result = dict(boxes=boxes, scores=scores, labels=labels)
    return result


print("Prediction Extraction")
preds = [get_format_pred(i, predictions, 0.0) for i in image_files]
print("Expected Extraction")
expected = [get_format_pred(i, valid_data, 0.0) for i in image_files]

Prediction Extraction
Expected Extraction


In [17]:
preds[0]

{'boxes': tensor([[ 7.3333e+02,  1.0013e+03,  1.3239e+02,  1.6801e+02],
         [ 2.1932e+03,  1.0334e+03,  7.6360e+01,  9.9060e+01],
         [ 7.6018e+02,  2.0583e+02,  8.2760e+01,  1.0425e+02],
         [ 1.3838e+03,  9.3542e+02,  7.6650e+01,  1.0031e+02],
         [ 1.5111e+03,  1.3883e+03,  7.5820e+01,  9.6400e+01],
         [ 3.8215e+02,  1.7691e+03,  7.9370e+01,  9.8730e+01],
         [ 2.3811e+03,  8.4556e+02,  7.6260e+01,  9.9750e+01],
         [ 1.4854e+03,  1.9080e+03,  7.6110e+01,  9.8160e+01],
         [ 3.6311e+02,  4.9899e+02,  8.0930e+01,  9.9650e+01],
         [ 1.0540e+03,  5.2273e+02,  7.7380e+01,  9.8970e+01],
         [ 2.2764e+03,  1.3469e+03,  7.7180e+01,  9.5720e+01],
         [ 2.4292e+03,  1.6844e+03,  7.8900e+01,  9.7710e+01],
         [ 2.2828e+03,  1.3414e+03,  7.6960e+01,  9.9180e+01],
         [ 2.4897e+03,  2.0669e+03,  8.0390e+01,  9.7700e+01],
         [ 1.3513e+03,  2.4917e+03,  8.2070e+01,  9.9540e+01],
         [ 2.1609e+03,  1.4849e+03,  7.4140e+0

In [18]:
expected[0]

{'boxes': tensor([[1744.,  925.,   79.,  116.],
         [2278.,  904.,  116.,  116.],
         [3060., 1141.,  100.,   95.],
         [1881., 1913.,   95.,  106.],
         [1702., 2489.,   85.,   90.],
         [2674., 1501.,  100.,   85.],
         [3023., 1675.,  106.,  111.],
         [2848., 1353.,   90.,  106.],
         [ 470., 1765.,  116.,  122.],
         [2943., 1226.,   95.,  106.],
         [2933.,  671.,   95.,   85.],
         [1311.,  534.,   90.,  100.],
         [ 254., 2003.,  100.,  111.],
         [ 941.,  206.,   90.,  106.],
         [2949., 1041.,   90.,  106.],
         [1649., 2669.,   90.,   95.],
         [ 460.,  523.,   90.,   85.],
         [ 925.,  988.,  169.,  206.],
         [1887., 1395.,   95.,   90.],
         [3012.,  772.,  106.,  100.],
         [2743., 1046.,   74.,   85.]], device='cuda:0'),
 'scores': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.], device='cuda:0'),
 'labels': tensor([0, 

In [19]:
from torchmetrics.detection import MeanAveragePrecision

metric = MeanAveragePrecision(
    iou_type="bbox",
    box_format="xywh",
    class_metrics=True,
    extended_summary=False,
    # backend="faster_coco_eval",
)


metric.update(preds=preds, target=expected)

from pprint import pprint

pprint(metric.compute())

{'classes': tensor(0, dtype=torch.int32),
 'map': tensor(0.0029),
 'map_50': tensor(0.0127),
 'map_75': tensor(0.0002),
 'map_large': tensor(0.0001),
 'map_medium': tensor(0.0070),
 'map_per_class': tensor(0.0029),
 'map_small': tensor(0.0027),
 'mar_1': tensor(0.0027),
 'mar_10': tensor(0.0207),
 'mar_100': tensor(0.0351),
 'mar_100_per_class': tensor(0.0351),
 'mar_large': tensor(0.0043),
 'mar_medium': tensor(0.0510),
 'mar_small': tensor(0.0282)}


In [20]:
expected[128]

{'boxes': tensor([[1277.,  127.,   85.,   70.],
         [1583.,  878.,   62.,   61.],
         [ 755.,  815.,   59.,   64.],
         [ 929.,  165.,   38.,   34.],
         [ 864.,  375.,   96.,   79.],
         [1430.,   66.,   26.,   25.],
         [ 473.,  513.,   32.,   38.],
         [1110.,  131.,   28.,   28.],
         [ 908.,  291.,   64.,   59.],
         [1394.,  600.,   28.,   26.],
         [1226.,  269.,   32.,   34.],
         [1432.,  110.,   30.,   32.],
         [ 653.,  815.,   30.,   32.],
         [1131.,  161.,   62.,   81.]], device='cuda:0'),
 'scores': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        device='cuda:0'),
 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0',
        dtype=torch.int32)}

In [26]:
preds[128]

{'boxes': tensor([[ 755.0400,  761.6900,   62.4200,   62.2600],
         [1582.9200,  823.0600,   57.0000,   55.6900],
         [ 917.6500,  273.1400,   56.5100,   58.0500],
         [1279.1600,  118.4900,   80.9700,   63.9600],
         [1130.1600,  157.1900,   60.6300,   73.6500],
         [ 857.2600,  357.1600,   74.3100,   69.1900],
         [ 741.5000,  134.5200,   55.4000,   56.7700],
         [1215.1200,  648.0500,   31.5900,   29.7500],
         [1225.4900,  253.3500,   32.2500,   31.1700],
         [ 711.8900,  292.8400,   29.9700,   29.1700],
         [ 628.6100,  391.2200,   34.7900,   33.5500],
         [ 990.2800,  449.2000,   42.1000,   36.2600],
         [ 205.7600,  262.8500,   33.1500,   29.5400],
         [ 688.0300,  601.5400,   59.5100,   47.4000],
         [ 472.7700,  484.5700,   34.0000,   32.1900],
         [1302.2400,  268.9500,   32.8600,   30.8600],
         [ 980.1200,  612.7300,   30.8800,   29.9900],
         [1109.7200,  120.5900,   30.6400,   28.5700],
 

In [29]:
import optuna
from optuna.trial import Trial

def objective(trial: Trial):
    thrs = [
        trial.suggest_float("thr0", 1.e-5, 1, log=True), trial.suggest_float("thr1", 1e-5, 1, log=True),
	]
    metric = MeanAveragePrecision(
        iou_type="bbox",
        box_format="xywh",
        class_metrics=True,
        extended_summary=False,
        # backend="faster_coco_eval",
    )
    preds = [get_format_pred(i, predictions, thrs) for i in image_files]
    expected = [get_format_pred(i, valid_data, thrs) for i in image_files]
    metric.update(preds=preds, target=expected)
    scores = metric.compute()
    return scores["map_50"]
    # if scores["map_50"] > best_score:
    #     best_score = scores["map_50"]
    #     best_thr = thrs
    #     print(best_thr, best_score)

In [37]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2024-11-11 14:19:59,794] A new study created in memory with name: no-name-ffea6863-740b-40ce-9e2b-a613bd9437bc
[I 2024-11-11 14:20:07,292] Trial 0 finished with value: 0.012680408544838428 and parameters: {'thr0': 0.00010064732376711895, 'thr1': 0.02663341861172616}. Best is trial 0 with value: 0.012680408544838428.
[I 2024-11-11 14:20:13,364] Trial 1 finished with value: 0.012680408544838428 and parameters: {'thr0': 0.07899989583700245, 'thr1': 1.4526013070565134e-05}. Best is trial 0 with value: 0.012680408544838428.
[I 2024-11-11 14:20:21,015] Trial 2 finished with value: 0.012680408544838428 and parameters: {'thr0': 0.00013498271160496572, 'thr1': 0.032147893745899055}. Best is trial 0 with value: 0.012680408544838428.
[I 2024-11-11 14:20:28,388] Trial 3 finished with value: 0.012680408544838428 and parameters: {'thr0': 1.0186313693978222e-05, 'thr1': 0.00011601632472458592}. Best is trial 0 with value: 0.012680408544838428.
[I 2024-11-11 14:20:36,026] Trial 4 finished with valu

In [38]:
study.best_params

{'thr0': 0.00010064732376711895, 'thr1': 0.02663341861172616}

In [39]:
study.best_value

0.012680408544838428

In [40]:
thr = list(study.best_params.values())

metric = MeanAveragePrecision(
    iou_type="bbox",
    box_format="xywh",
    class_metrics=True,
    extended_summary=False,
    backend="faster_coco_eval",
)
preds = [get_format_pred(i, predictions, thr) for i in image_files]
expected = [get_format_pred(i, valid_data, thr) for i in image_files]
metric.update(preds=preds, target=expected)
scores = metric.compute()

In [41]:
scores

{'map': tensor(0.0029),
 'map_50': tensor(0.0127),
 'map_75': tensor(0.0002),
 'map_small': tensor(0.0027),
 'map_medium': tensor(0.0070),
 'map_large': tensor(0.0001),
 'mar_1': tensor(0.0027),
 'mar_10': tensor(0.0207),
 'mar_100': tensor(0.0351),
 'mar_small': tensor(0.0282),
 'mar_medium': tensor(0.0510),
 'mar_large': tensor(0.0043),
 'map_per_class': tensor(0.0029),
 'mar_100_per_class': tensor(0.0351),
 'classes': tensor(0, dtype=torch.int32)}